# Dogs vs Cats classification using simple linear regression
By: Andrés M. Castillo

Warning!! Real life is not thaat simple. I've selected the images to get this little introductory example working for you.

Objective: Build a classification model that predicts the animal(cat or dog) in the picture. The purpose of this example is to illustrate the general machine learning setup and feature engeeniering process. For the ones who don't not know, bellow I show a picture of a real cat and a real dog

Cat | Dog
- | - 
<img src="data/cats/cat0.jpeg"> |  <img src="data/dogs/dog0.jpeg">

 Now lets start to see how many pictures we have and what is its format. But before let's install some libraries that we'll need during this example
 


In [4]:
!conda config --set allow_conda_downgrades true
!conda install conda=4.6.14 -y
!conda install -y -c conda-forge opencv

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: //anaconda3

  added / updated specs:
    - conda=4.6.14


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.6.16          |           py37_0         151 KB
    conda-4.6.14               |           py37_0         1.9 MB
    ------------------------------------------------------------
                                           Total:         2.1 MB

The following packages will be UPDATED:

  anaconda                                    custom-py37_1 --> 2019.07-py37_0

The following packages will be DOWNGRADED:

  bokeh                                        1.3.4-py37_0 --> 1.2.0-py37_0
  ca-certificates                               2019.

In [8]:
!conda install -y numpy --force-reinstall

Solving environment: done

## Package Plan ##

  environment location: //anaconda3

  added / updated specs:
    - numpy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.6.16          |           py37_1         155 KB
    conda-4.7.11               |           py37_0         3.0 MB
    numpy-1.16.4               |   py37h926163e_0          48 KB
    ------------------------------------------------------------
                                           Total:         3.2 MB

The following packages will be UPDATED:

  openssl            conda-forge::openssl-1.1.1c-h01d97ff_0 --> pkgs/main::openssl-1.1.1c-h1de35cc_1

The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates    conda-forge::ca-certificates-2019.6.1~ --> pkgs/main::ca-certificates-2019.5.15-1
  certifi                                       conda-forge --> pkgs/main
  conda       

In [ ]:
!ls -alh data/*/*

# Data preparation / Preprocessing

In this step we must load our dataset into main memory to start playing with it. 

False. In this little example perhaps that is possible, but in real life you wont be able to load all your images into main memory.
And even if the dataset fits into memory, it is not recomended in any case to do so. In a real image analysis you must scale all the images to fit a given size. For the kind of analysis we are goin to peform, we can extract the features directly.

Let's understand the feature extraction for a single image

In [5]:
# import the necessary packages
from matplotlib import pyplot as plt
import numpy as np
import argparse
import cv2
 
# construct the argument parser and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-i", "--image", required = True, help = "data/cats/cat0.jpeg")
args = vars(ap.parse_args())
 
# load the image and show it
image = cv2.imread(args["image"])
cv2.imshow("image", image)

ImportError: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the multiarray numpy extension module failed.  Most
likely you are trying to import a failed build of numpy.
Here is how to proceed:
- If you're working with a numpy git repository, try `git clean -xdf`
  (removes all files not under version control) and rebuild numpy.
- If you are simply trying to use the numpy version that you have installed:
  your installation is broken - please reinstall numpy.
- If you have already reinstalled and that did not fix the problem, then:
  1. Check that you are using the Python you expect (you're using //anaconda3/bin/python),
     and that you have no directories in your PATH or PYTHONPATH that can
     interfere with the Python and numpy versions you're trying to use.
  2. If (1) looks fine, you can open a new issue at
     https://github.com/numpy/numpy/issues.  Please include details on:
     - how you installed Python
     - how you installed numpy
     -

In [ ]:
# convert the image to grayscale and create a histogram
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
cv2.imshow("gray", gray)
hist = cv2.calcHist([gray], [0], None, [256], [0, 256])
plt.figure()
plt.title("Grayscale Histogram")
plt.xlabel("Bins")
plt.ylabel("# of Pixels")
plt.plot(hist)
plt.xlim([0, 256])

A call to plt.show() displays:

In [6]:
plt.show()

NameError: name 'plt' is not defined

In [ ]:
Now, lets compute a flattened color histogram

In [ ]:
# grab the image channels, initialize the tuple of colors,
# the figure and the flattened feature vector
chans = cv2.split(image)
colors = ("b", "g", "r")
plt.figure()
plt.title("'Flattened' Color Histogram")
plt.xlabel("Bins")
plt.ylabel("# of Pixels")
features = []
 
# loop over the image channels
for (chan, color) in zip(chans, colors):
	# create a histogram for the current channel and
	# concatenate the resulting histograms for each
	# channel
	hist = cv2.calcHist([chan], [0], None, [256], [0, 256])
	features.extend(hist)
 
	# plot the histogram
	plt.plot(hist, color = color)
	plt.xlim([0, 256])
 
# here we are simply showing the dimensionality of the
# flattened color histogram 256 bins for each channel
# x 3 channels = 768 total values -- in practice, we would
# normally not use 256 bins for each channel, a choice
# between 32-96 bins are normally used, but this tends
# to be application dependent
print "flattened feature vector size: %d" % (np.array(features).flatten().shape)